<a href="https://colab.research.google.com/github/GawainGan/Stock-Markets-Analytics/blob/main/Code/Module_2_Web_Scraping_and_IPO_Data_Analysis_Strategies_for_Optimal_Growth_and_Profitability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this work, we utilized web scraping techniques to extract IPO data from various online sources and applied several data analysis strategies to evaluate financial growth and profitability. The tasks included:

1. **IPO Filings Web Scraping and Data Processing**: Extracted and processed IPO filings data to calculate the total sum of 2023 filings occurring on Fridays.
2. **IPOs "Fixed Days Hold" Strategy**: Identified the optimal holding period (1 to 30 days) for maximizing the 75% quantile growth among IPO stocks.
3. **Growth Comparison Between Large and Largest Stocks**: Analyzed the share of days when large stocks outperformed the largest stocks in terms of 7-day growth.
4. **Technical Indicators Strategy Using CCI**: Evaluated the gross profit from trading based on the Commodity Channel Index (CCI) over a 10-year period, considering both gross profit and the impact of trading fees.

Through these analyses, we aimed to derive actionable insights and develop effective trading strategies based on historical IPO data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install yfinance
# !pip install pandas_datareader

### package

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date
from datetime import datetime
from datetime import timedelta

# for graphs
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


# 1.IPO Filings Web Scraping and Data Processing

**Task**: Calculate the total sum ($m) of 2023 IPO filings that occurred on Fridays.

**Steps**:
1. **Web Scraping**: Use the provided code snippet to scrape data from [stockanalysis.com/ipos/filings/](https://stockanalysis.com/ipos/filings/).
2. **Data Conversion**:
   - Convert 'Filing Date' to datetime.
   - Convert 'Shares Offered' to float64, replacing '-' with NaNs.
3. **Data Processing**:
   - Create a new field 'Avg_price' based on 'Price Range':
     - NaN if no price.
     - Use single price if one number is provided.
     - Use the average if a range is provided.
4. **Calculate Values**:
   - Define 'Shares_offered_value' as 'Shares Offered' multiplied by 'Avg_price', or NaN if either is missing.
5. **Analysis**:
   - Filter for 2023 filings on Fridays (Date.dt.dayofweek() == 4).
   - Sum the 'Shares_offered_value' for these records, returning the total in millions of USD (rounded to the nearest integer).

Note: There should be 32 records in total, with 25 non-null values.

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}
# Define a new field 'Avg_price' based on the "Price Range",
# which equals to NaN if no price is specified, to the price (if only one number is provided),
# or to the average of 2 prices (if a range is given).

filling_url = 'https://stockanalysis.com/ipos/filings/'
response = requests.get(filling_url, headers=headers)

ipo_fillings_dfs = pd.read_html(response.text)

ipo_fillings_dfs[0]
filing_2023 = ipo_fillings_dfs[0][ipo_fillings_dfs[0]['Filing Date'].str.contains('2023')]

def calculate_avg_price(price_range):
    # case 1: if NaN
    if price_range == '-':
        return np.nan
    # case 2: if range
    elif '-' in price_range:
        prices = price_range.split('-')
        price1 = float(prices[0].strip().replace('$', ''))
        price2 = float(prices[1].strip().replace('$', ''))
        return (price1 + price2) / 2
    # case 3: if single price
    else:
        return float(price_range.strip().replace('$', ''))

filing_2023['Avg_price'] = filing_2023['Price Range'].apply(calculate_avg_price)


filing_2023.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
49,"Dec 29, 2023",LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000,4.0
50,"Dec 29, 2023",EPSM,Epsium Enterprise Limited,-,-,NaN
51,"Dec 28, 2023",ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667,7.5
52,"Dec 22, 2023",CHLW,Chun Hui Le Wan International Holding Group Ltd,-,-,NaN
53,"Dec 22, 2023",LZMH,LZ Technology Holdings Limited,-,-,NaN


In [ ]:
# convert 'Filing Date' to datetime,format like: .apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d'))
# convert Shares Offered to numeric
filing_2023['Filing Date'] = filing_2023['Filing Date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d'))
filing_2023['Filing Date'] = pd.to_datetime(filing_2023['Filing Date'])
filing_2023['Shares Offered'] = pd.to_numeric(filing_2023['Shares Offered'].str.replace(',', ''), errors='coerce')
filing_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117 entries, 49 to 165
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     117 non-null    datetime64[ns]
 1   Symbol          117 non-null    object        
 2   Company Name    117 non-null    object        
 3   Price Range     117 non-null    object        
 4   Shares Offered  85 non-null     float64       
 5   Avg_price       89 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 10.5+ KB


In [ ]:
#  Define a column "Shares_offered_value",
# which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
filing_2023['Shares_offered_value'] = filing_2023['Shares Offered'] * filing_2023['Avg_price']
filing_2023.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
49,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.0,4800000.0
50,2023-12-29,EPSM,Epsium Enterprise Limited,-,NaN,NaN,NaN
51,2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667.0,7.5,8000002.5
52,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,NaN,NaN,NaN
53,2023-12-22,LZMH,LZ Technology Holdings Limited,-,NaN,NaN,NaN


In [ ]:
# order by 'Filing Date'
filing_2023.sort_values(by='Filing Date', inplace=True)
# reset the index
filing_2023.reset_index(drop=True, inplace=True)
filing_2023.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2023-01-10,FDAN,FD Technology Inc.,$4.00 - $6.00,3500000.0,5.0,17500000.0
1,2023-01-13,RVGO,"RVeloCITY, Inc.",$4.00 - $5.00,3750000.0,4.5,16875000.0
2,2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,NaN,NaN,NaN
3,2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000.0,5.0,16500000.0
4,2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000.0,4.5,8437500.0


In [ ]:
# Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4).
# You should see 32 records in total, 25 of it is not null.

friday_filings = filing_2023[filing_2023['Filing Date'].dt.dayofweek == 4]
len(friday_filings)

32

In [ ]:
# save the total sum in $m (millions of USD, closest INTEGER number) for all friday_filings
total_friday_filings = friday_filings['Shares_offered_value'].sum()
total_friday_filings = int(round(total_friday_filings / 1_000_000))
print("Total sum in for all friday_filings: $m", total_friday_filings, "USD")

Total sum in for all friday_filings: $m 286 USD


# 2: IPOs "Fixed days hold" strategy

**Task**: Find the optimal number of days \( X \) (between 1 and 30) where the 75% quantile growth is the highest.

**Steps**:
1. **Web Scraping**: Reuse the code snippet to scrape IPO lists from [stockanalysis.com/ipos/2023/](https://stockanalysis.com/ipos/2023/) and [stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/).
2. **Filter Data**:
   - Include IPOs with dates before March 1, 2024.
   - Exclude the ticker 'RYZB'.
3. **Retrieve OHLCV Prices**:
   - Adjust symbol names if necessary to match Yahoo Finance tickers.
   - Collect daily OHLCV prices for each ticker.
4. **Calculate Growth**:
   - For each stock, calculate the growth from the adjusted close price on the IPO date to the adjusted close price after \( X \) days, for \( X \) ranging from 1 to 30 days.
5. **Quantile Analysis**:
   - Generate 30 additional columns for growth after \( X \) days (growth_future_1d ... growth_future_30d).
   - Compute the 75% quantile for growth for each \( X \).
   - Identify the \( X \) value with the highest 75% quantile growth.

**Recommendation**:
1. Ensure that the mean and median investment returns are negative for most \( X \) values, indicating the strategy benefits a "lucky" investor who falls in the top 25%.
2. Evaluate whether pursuing this strategy is advisable based on the overall distribution of returns and the optimal \( X \) identified.

### Summary of the Strategy:
The objective is to determine the optimal holding period \( X \) days that maximizes the 75th percentile of investment returns among 185 IPO stocks. Analyze the data to see if this strategy can provide a significant advantage to an investor who falls in the top quartile.

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url_2023 = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url_2023, headers=headers)
ipos_2023 = pd.read_html(response.text)
df_ipos_2023 = ipos_2023[0]

url_2024 = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url_2024, headers=headers)
ipos_2024 = pd.read_html(response.text)
df_ipos_2024 = ipos_2024[0]

df_ipos_2023['IPO Date']=pd.to_datetime(df_ipos_2023['IPO Date'],errors='raise',format ='mixed')
df_ipos_2024['IPO Date']=pd.to_datetime(df_ipos_2024['IPO Date'],errors='raise',format='mixed')

df_ipos_2023=df_ipos_2023.loc[df_ipos_2023['IPO Date']<"2024-03-01"]
df_ipos_2024=df_ipos_2024.loc[df_ipos_2024['IPO Date']<"2024-03-01"]

stock_lst=list(set(pd.concat([df_ipos_2023['Symbol'],df_ipos_2024['Symbol']],axis=0)))
# remove 'RYZB'
stock_lst.remove('RYZB')
print(len(stock_lst))

184


In [ ]:
end_dt = date(2024,3,1)
df_OHLCV=yf.download(stock_lst,end=end_dt,progress=True)

df_copy=df_OHLCV.copy()
df_adj_close=df_copy['Adj Close']
df_adj_close.tail(30)

[*********************100%%**********************]  184 of 184 completed
ERROR:yfinance:
4 Failed downloads:
ERROR:yfinance:['DYCQ', 'JVSA', 'LEGT']: Exception("%ticker%: Data doesn't exist for startDate = -1406851200, endDate = 1709269200")
ERROR:yfinance:['PTHR']: Exception('%ticker%: No timezone found, symbol may be delisted')


Ticker,AACT,AESI,AFJK,AHR,AITR,AIXI,ALCY,ANL,ANRO,ANSC,...,VHAI,VSME,VTMX,WBUY,WLGS,WRNT,YGFGF,YIBO,ZJYL,ZKH
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-18,10.465,16.587967,NaN,NaN,10.200,1.79,10.490,9.050,NaN,10.10,...,NaN,0.369,36.618332,0.399,0.700,0.350,1.530,NaN,5.2575,16.730000
2024-01-19,10.470,16.172773,NaN,NaN,10.210,1.61,10.490,8.989,NaN,10.11,...,NaN,0.330,37.434284,0.380,0.584,0.333,1.570,NaN,5.2500,16.100000
2024-01-22,10.480,16.933960,NaN,NaN,10.210,1.58,10.500,9.120,NaN,10.13,...,NaN,0.319,37.334778,0.395,0.590,0.330,1.420,NaN,5.3500,16.200001
2024-01-23,10.480,16.775791,10.160,NaN,10.210,1.69,10.600,9.120,NaN,10.13,...,NaN,0.328,37.205418,0.450,0.610,0.350,1.270,NaN,8.3450,16.809999
2024-01-24,10.480,16.627508,10.160,NaN,10.200,1.82,10.510,9.213,NaN,10.12,...,NaN,0.649,37.444237,0.460,0.600,0.348,1.250,NaN,10.5925,18.520000
2024-01-25,10.480,17.072359,10.170,NaN,10.200,1.88,10.560,9.213,NaN,10.12,...,NaN,0.487,38.568653,0.467,0.620,0.350,1.060,2.790,13.8055,19.040001
2024-01-26,10.490,17.240414,10.170,NaN,10.210,1.90,10.560,9.213,NaN,10.12,...,NaN,0.650,38.528854,0.445,0.660,0.345,0.858,2.720,8.9000,17.150000
2024-01-29,10.500,17.368925,10.180,NaN,10.210,1.90,10.520,9.130,NaN,10.12,...,NaN,0.599,38.260185,0.464,0.659,0.344,0.685,2.600,9.1495,17.750000
2024-01-30,10.490,17.527094,10.170,NaN,10.200,1.94,10.525,9.130,NaN,10.13,...,NaN,0.478,37.941769,0.428,0.790,0.330,0.731,2.660,10.5200,18.120001


In [ ]:
sub = df_adj_close[list(df_adj_close.columns)[0]]
sub_df = pd.DataFrame(sub)
sub_df['future_growth_1d'] = sub_df.shift(-1)/sub_df
sub_df.dropna(inplace=True)
sub_df

,AACT,future_growth_1d
Date,,
2023-06-13,10.11,0.999011
2023-06-14,10.10,1.000990
2023-06-15,10.11,1.000000
2023-06-16,10.11,1.000989
2023-06-20,10.12,1.000000
...,...,...
2024-02-22,10.55,1.000000
2024-02-23,10.55,0.999052
2024-02-26,10.54,1.000949


In [ ]:
def return_sub_df_future_growth(df, stock_name):
    # args's df isdfdf_adj_close

    # Select the first stock from the DataFrame (example)
    sub = df.loc[:, stock_name]
    sub_df = pd.DataFrame(sub)

    # Calculate future growth for 1 to 30 days using a loop
    for days in range(1, 31):
        sub_df[f'future_growth_{days}d'] = sub_df[sub.name].shift(-days) / sub_df[sub.name]

    # Drop rows with any NaN values (these will be the last rows up to the maximum shift of 30 days)
    sub_df.dropna(inplace=True)
    # keep only first row
    sub_df = sub_df.head(1)
    # drop index and set new index as the name of the stock
    sub_df.reset_index(inplace=True)
    # drop the first two colmn
    # and insert the stock name as the first column
    sub_df.insert(0, 'Stock', sub.name)
    sub_df.drop(columns=['Date', sub.name], inplace=True)

    # Display the DataFrame
    # sub_df.head()
    return sub_df

df_test = return_sub_df_future_growth(df_adj_close, 'AACT')
df_test

,Stock,future_growth_1d,future_growth_2d,future_growth_3d,future_growth_4d,future_growth_5d,future_growth_6d,future_growth_7d,future_growth_8d,future_growth_9d,...,future_growth_21d,future_growth_22d,future_growth_23d,future_growth_24d,future_growth_25d,future_growth_26d,future_growth_27d,future_growth_28d,future_growth_29d,future_growth_30d
0,AACT,0.999011,1.0,1.0,1.000989,1.000989,1.000989,1.002967,1.002967,1.003956,...,1.005539,1.004946,1.004946,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913


In [ ]:
def return_the_whole_df_with_future_growth(df):
    # initialize DataFrame for all stock data
    df_merge = pd.DataFrame()
    col_list = list(df.columns)
    print(len(col_list))
    # iterative for each stock
    for col in col_list:
        # get their future growth
        sub_df = return_sub_df_future_growth(df, col)
        # save in one df
        df_merge = pd.concat([df_merge, sub_df], ignore_index=True)

    return df_merge

df_tmp = return_the_whole_df_with_future_growth(df_adj_close)
# sort by the first column
df_tmp.sort_values(by='Stock', inplace=True)
df_tmp.reset_index(drop=True, inplace=True)
df_tmp

184


,Stock,future_growth_1d,future_growth_2d,future_growth_3d,future_growth_4d,future_growth_5d,future_growth_6d,future_growth_7d,future_growth_8d,future_growth_9d,...,future_growth_21d,future_growth_22d,future_growth_23d,future_growth_24d,future_growth_25d,future_growth_26d,future_growth_27d,future_growth_28d,future_growth_29d,future_growth_30d
0,AACT,0.999011,1.000000,1.000000,1.000989,1.000989,1.000989,1.002967,1.002967,1.003956,...,1.005539,1.004946,1.004946,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913
1,AESI,0.973451,0.943363,0.979941,0.946903,0.951032,0.939823,0.949853,0.965192,0.968142,...,1.023009,1.049557,1.050737,1.058997,1.076106,1.074926,1.057817,1.066077,1.056637,1.072566
2,AITR,1.000986,1.003945,1.003945,1.004043,1.004931,1.004931,1.004931,1.005424,1.004931,...,1.006903,1.006903,1.006903,1.007890,1.008876,1.007890,1.007692,1.007692,1.008383,1.008876
3,AIXI,1.117040,1.098107,1.025818,0.987952,0.965577,0.970740,0.963855,1.084337,1.020654,...,0.934596,0.932874,0.984509,1.086059,1.056627,1.048193,1.006885,0.984165,0.979346,0.946300
4,ALCY,0.996094,0.996094,0.996094,1.000000,1.000000,0.994141,0.994141,0.994141,0.994141,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.001953,1.000000,1.002930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,WLGS,1.012903,0.874194,0.738710,0.583871,0.593548,0.716129,0.754839,0.800000,0.761290,...,0.564516,0.587097,0.619355,0.545161,0.525806,0.470968,0.448387,0.548387,0.532258,0.580645
149,WRNT,0.788372,0.572093,0.508139,0.441860,0.372093,0.402326,0.369767,0.365116,0.318605,...,0.241860,0.223256,0.226744,0.209302,0.244186,0.217674,0.209302,0.209302,0.213953,0.225814
150,YGFGF,0.975064,0.976864,0.997686,1.002571,0.940874,0.866324,0.843188,0.843188,0.832905,...,0.950386,0.904884,1.002571,0.866324,0.868895,0.750643,0.758355,0.754499,0.830334,0.943445
151,ZJYL,0.997987,0.962264,0.937107,0.943396,0.916981,0.940881,0.924528,0.922013,0.928302,...,0.899371,0.959748,1.207547,0.739623,0.691824,0.704403,0.716981,0.704403,0.716730,0.703145


In [ ]:
df_tmp.describe().loc['75%']

future_growth_1d     1.011287
future_growth_2d     1.012168
future_growth_3d     1.006689
future_growth_4d     1.004965
future_growth_5d     1.004965
future_growth_6d     1.004931
future_growth_7d     1.004931
future_growth_8d     1.005424
future_growth_9d     1.006897
future_growth_10d    1.006890
future_growth_11d    1.008008
future_growth_12d    1.010204
future_growth_13d    1.018456
future_growth_14d    1.012258
future_growth_15d    1.012884
future_growth_16d    1.012884
future_growth_17d    1.010834
future_growth_18d    1.011823
future_growth_19d    1.014706
future_growth_20d    1.014286
future_growth_21d    1.011811
future_growth_22d    1.013726
future_growth_23d    1.015625
future_growth_24d    1.020625
future_growth_25d    1.020089
future_growth_26d    1.024490
future_growth_27d    1.023638
future_growth_28d    1.027273
future_growth_29d    1.025503
future_growth_30d    1.021990
Name: 75%, dtype: float64

In [ ]:
idmax = df_tmp.describe().loc['75%'].idxmax()
max_75 = df_tmp.describe().loc['75%'].max()
print(f'The stock with the highest 75% quantile of future growth is {idmax} with a value of {max_75}')

The stock with the highest 75% quantile of future growth is future_growth_28d with a value of 1.0272727446122603


# 3: Is Growth Concentrated in the Largest Stocks?

**Task**: Calculate the percentage of days when large stocks outperform the largest stocks in terms of 7-day growth.

**Steps**:
1. **Retrieve Data**:
   - Use Code Snippet 7 to obtain OHLCV data for 33 largest stocks and 32 large stocks for 10 full years (2014-01-01 to 2023-12-31), plus data for 7 days before 2014-01-01 to correctly calculate initial growth_7d.
   
2. **Calculate 7-Day Growth**:
   - For each stock, compute the growth over 7 periods (growth_7d).

3. **Organize Data**:
   - Create a dataframe that includes the date, stock category (LARGE or LARGEST), and growth_7d.
   - Use `pandas.pivot_table()` to create a flattened table with separate columns for LARGE and LARGEST growth_7d values.

4. **Compare Performance**:
   - Determine the number of days where the average daily growth_7d of the LARGE group exceeds that of the LARGEST group.
   - Calculate the percentage of these days out of the total number of trading days (2595).

**Result**:
- Convert the ratio of outperforming days to a percentage (rounded to the nearest integer).

**Analysis**:
- Determine the share of days when large stocks outperform the largest stocks.
- Use this analysis to consider if expanding the dataset with more stocks can lead to higher growth opportunities.

In [ ]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

LARGEST_STOCKS.sort()
LARGE_STOCKS.sort()
print(f"The qty largest stocks {len(LARGEST_STOCKS)}")
print(f"The qty large stocks {len(LARGE_STOCKS)}")

The qty largest stocks 33
The qty large stocks 34


In [ ]:
start_date=date(2014,1,1)
end_date=date(2023,12,31)
start_date=start_date-timedelta(days=12)
df_LARGEST_STOCK = yf.download(LARGEST_STOCKS,start_date,end_date,progress=True)['Adj Close']

df_LARGE_STOCK = yf.download(LARGE_STOCKS,start_date,end_date,progress=True)['Adj Close']

df_LARGEST_STOCK['growth_7d']=(df_LARGEST_STOCK/df_LARGEST_STOCK.shift(7)).mean(axis=1)
df_LARGEST_STOCK['ticket_category']='LARGEST'
df_LARGEST_STOCK=df_LARGEST_STOCK['2014-01-01':]

[*********************100%%**********************]  33 of 33 completed
[*********************100%%**********************]  33 of 33 completed


In [ ]:
df_LARGEST_STOCK.head()

Ticker,AAPL,ACN,AMZN,ASML,AVGO,BHARTIARTL.NS,BRK-B,CDI.PA,GOOG,HDB,...,RELIANCE.NS,RMS.PA,SAP,SBIN.NS,SIE.DE,TCS.NS,TTE,V,growth_7d,ticket_category
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,NaN,NaN,NaN,NaN,NaN,288.068909,NaN,NaN,NaN,NaN,...,379.456085,NaN,NaN,160.800262,NaN,872.926453,NaN,NaN,1.011797,LARGEST
2014-01-02,17.296658,67.617805,19.898500,83.166672,39.869331,280.350128,117.500000,108.007118,27.724083,15.844663,...,373.564392,237.023544,71.747696,158.755020,61.293823,878.480164,32.874443,51.426998,0.998189,LARGEST
2014-01-03,16.916718,67.842842,19.822001,82.878525,40.020775,280.904602,117.570000,108.086693,27.521841,15.844663,...,368.974945,237.974182,71.257607,156.664246,61.609623,900.857605,32.622696,51.461906,0.996692,LARGEST
2014-01-06,17.008968,67.126083,19.681499,81.176636,39.748169,280.904602,116.279999,105.937698,27.828691,16.172998,...,365.004456,233.266281,71.333641,153.794510,61.417664,907.911804,32.524197,51.152431,0.990353,LARGEST
2014-01-07,16.887331,67.942863,19.901501,80.492310,40.126801,282.908875,116.190002,105.062180,28.365179,16.158928,...,359.518402,229.916443,71.130836,151.357529,61.727264,894.351074,32.617226,51.543335,0.987984,LARGEST


In [ ]:
df_LARGE_STOCK['growth_7d']=(df_LARGE_STOCK/df_LARGE_STOCK.shift(7)).mean(axis=1)
df_LARGE_STOCK['ticket_category']='LARGE'
df_LARGE_STOCK=df_LARGE_STOCK['2014-01-01':]
df_LARGE_STOCK.head()

Ticker,ADANIENT.NS,AI.PA,AIR.PA,ALV.DE,BAJFINANCE.NS,BUD,CDI.PA,COST,DTE.DE,EL.PA,...,SU.PA,SUNPHARMA.NS,TATAMOTORS.NS,TITAN.NS,TSLA,UNH,WMT,XOM,growth_7d,ticket_category
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,37.967480,NaN,NaN,NaN,152.003906,NaN,NaN,NaN,NaN,NaN,...,NaN,533.223511,368.095581,220.363159,NaN,NaN,NaN,NaN,1.009288,LARGE
2014-01-02,36.074402,53.805916,48.513958,75.411369,151.897461,80.872635,108.007118,94.629097,8.384732,64.263039,...,48.639286,534.620972,365.543121,216.114380,10.006667,63.748573,21.187386,64.282166,0.999349,LARGE
2014-01-03,35.085480,54.125713,48.385277,75.264244,150.571289,81.021027,108.086693,94.211388,8.481505,64.717667,...,48.678345,540.071045,356.069519,217.116913,9.970667,64.201645,21.117582,64.127495,0.999633,LARGE
2014-01-06,35.608192,53.033066,47.553112,75.558464,150.179260,80.083824,105.937698,93.496521,8.426206,65.357521,...,48.897087,548.036560,360.241821,217.498810,9.800000,63.466465,20.999439,64.224174,0.990707,LARGE
2014-01-07,34.181332,52.206924,47.964901,76.558868,150.430939,81.598969,105.062180,93.062790,8.467681,65.753220,...,48.358040,553.579773,358.425629,215.827957,9.957333,65.407028,21.063879,65.132820,0.990938,LARGE


In [ ]:
df_LARGE_STOCK.reset_index(inplace=True)
df_LARGEST_STOCK.reset_index(inplace=True)

In [ ]:
df_LARGE_STOCK.head()

Ticker,Date,ADANIENT.NS,AI.PA,AIR.PA,ALV.DE,BAJFINANCE.NS,BUD,CDI.PA,COST,DTE.DE,...,SU.PA,SUNPHARMA.NS,TATAMOTORS.NS,TITAN.NS,TSLA,UNH,WMT,XOM,growth_7d,ticket_category
0,2014-01-01,37.967480,NaN,NaN,NaN,152.003906,NaN,NaN,NaN,NaN,...,NaN,533.223511,368.095581,220.363159,NaN,NaN,NaN,NaN,1.009288,LARGE
1,2014-01-02,36.074402,53.805916,48.513958,75.411369,151.897461,80.872635,108.007118,94.629097,8.384732,...,48.639286,534.620972,365.543121,216.114380,10.006667,63.748573,21.187386,64.282166,0.999349,LARGE
2,2014-01-03,35.085480,54.125713,48.385277,75.264244,150.571289,81.021027,108.086693,94.211388,8.481505,...,48.678345,540.071045,356.069519,217.116913,9.970667,64.201645,21.117582,64.127495,0.999633,LARGE
3,2014-01-06,35.608192,53.033066,47.553112,75.558464,150.179260,80.083824,105.937698,93.496521,8.426206,...,48.897087,548.036560,360.241821,217.498810,9.800000,63.466465,20.999439,64.224174,0.990707,LARGE
4,2014-01-07,34.181332,52.206924,47.964901,76.558868,150.430939,81.598969,105.062180,93.062790,8.467681,...,48.358040,553.579773,358.425629,215.827957,9.957333,65.407028,21.063879,65.132820,0.990938,LARGE


In [ ]:
# keep Date, growth_7d and ticker_category for both dataframes
df_LARGE_STOCK = df_LARGE_STOCK[['Date', 'growth_7d', 'ticket_category']]
df_LARGEST_STOCK = df_LARGEST_STOCK[['Date', 'growth_7d', 'ticket_category']]

df_LARGE_STOCK

Ticker,Date,growth_7d,ticket_category
0,2014-01-01,1.009288,LARGE
1,2014-01-02,0.999349,LARGE
2,2014-01-03,0.999633,LARGE
3,2014-01-06,0.990707,LARGE
4,2014-01-07,0.990938,LARGE
...,...,...,...
2590,2023-12-22,1.003809,LARGE
2591,2023-12-26,1.014441,LARGE
2592,2023-12-27,1.003292,LARGE
2593,2023-12-28,1.001857,LARGE


In [ ]:
df_LARGEST_STOCK

Ticker,Date,growth_7d,ticket_category
0,2014-01-01,1.011797,LARGEST
1,2014-01-02,0.998189,LARGEST
2,2014-01-03,0.996692,LARGEST
3,2014-01-06,0.990353,LARGEST
4,2014-01-07,0.987984,LARGEST
...,...,...,...
2590,2023-12-22,1.013788,LARGEST
2591,2023-12-26,1.014900,LARGEST
2592,2023-12-27,1.013800,LARGEST
2593,2023-12-28,1.009875,LARGEST


In [ ]:
out_form_day = (df_LARGE_STOCK['growth_7d'] > df_LARGEST_STOCK['growth_7d'])
out_form_number = out_form_day.sum()
total_day = len(df_LARGE_STOCK)
per_out_form = out_form_number / total_day
print(f'The percentage of days when large stocks outperformed largest stocks is {per_out_form:.2%}')

The percentage of days when large stocks outperformed largest stocks is 46.63%


# 4: Trying Another Technical Indicators strategy

**Task**: Calculate the total gross profit from trading using the Commodity Channel Index (CCI) strategy, assuming no fees, and determine profitability after fees.

**Steps**:
1. **Data Preparation**:
   - Run the entire Colab notebook to obtain the full DataFrame.
   - Truncate the data to the last 10 years (2014-01-01 to 2023-12-31).

2. **CCI Strategy**:
   - Use a high threshold of 200 for the CCI indicator.
   - Trade only on Fridays (Date.dt.dayofweek() == 4).
   - Invest $1000 each time CCI > 200 at the Adj. Close price and hold for 5 trading days, then sell at the Adj. Close price.

3. **Calculate Gross Profit**:
   - For each trade, compute the profit: (Selling price - Buying price) * $1000 / Buying price.
   - Sum the profits over all trades to get the total gross profit.
   - Convert the total gross profit to thousands of dollars and round to the nearest integer.

4. **Fee Calculation** (Additional):
   - Use the [DEGIRO fee calculator](https://www.degiro.ie/fees/calculator) to estimate fees for 460 trades per year with a transaction amount of €1000.
   - Determine if the strategy remains profitable after accounting for these fees.

**Expected Result**:
- The total gross profit in thousands of dollars (rounded to the nearest integer) from the CCI strategy over the 10-year period.
- An analysis of profitability after including trading fees, indicating whether the strategy is still viable.

### Summary of the Strategy:
The objective is to use the CCI indicator to identify trading opportunities and calculate the resulting gross profit over 10 years. Additionally, consider the impact of trading fees on the overall profitability of the strategy. This analysis will help determine the effectiveness and viability of the CCI-based trading strategy.

In [ ]:
# !pip install fastparquet

In [ ]:
path = './stocks_df_combined_trunc_2014_2023.parquet.brotli'
m16_df = pd.read_parquet(path)
m16_df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_buy_T = m16_df[(m16_df['cci'] > 200 ) & (m16_df['Weekday'] == 4)].copy()
df_buy_T = df_buy_T[['Date','Adj Close_x','Ticker','cci','Weekday']]

df_buy_T.head()

,Date,Adj Close_x,Ticker,cci,Weekday
7113,2014-05-30,34.912762,MSFT,203.518798,4
7181,2014-09-05,39.395618,MSFT,257.013330,4
7340,2015-04-24,41.630741,MSFT,340.009871,4
7452,2015-10-02,40.151123,MSFT,206.125828,4
7467,2015-10-23,46.583046,MSFT,431.107024,4


In [ ]:
# Add 'buy' column or reset it to 0 initially
df_buy_T['buy'] = 0  # Ensure that 'buy' column exists and reset all to 0

# Use np.where to conditionally update the 'buy' column
df_buy_T['buy'] = np.where((df_buy_T['cci'] > 200) & (df_buy_T['Weekday'] == 4), 1, df_buy_T['buy'])

total_buy_T = len(df_buy_T[df_buy_T['buy']==1])
print("total counts of Buy== 1:", total_buy_T)

# Show the updated DataFrame
df_buy_T.head()

total counts of Buy== 1: 460


,Date,Adj Close_x,Ticker,cci,Weekday,buy
7113,2014-05-30,34.912762,MSFT,203.518798,4,1
7181,2014-09-05,39.395618,MSFT,257.013330,4,1
7340,2015-04-24,41.630741,MSFT,340.009871,4,1
7452,2015-10-02,40.151123,MSFT,206.125828,4,1
7467,2015-10-23,46.583046,MSFT,431.107024,4,1


In [ ]:
# Function to find the next valid trading day (5 days later)
def find_next_trading_day(current_date, delta=5):
    # Find the date 5 days later
    target_date = current_date + timedelta(days=delta)

    return target_date

In [ ]:
# testing
test_date = datetime(2014, 9, 5)
test_ticker = 'MSFT'
test_date_after_5 = find_next_trading_day(test_date)

print(f"Test Date: {test_date}")
print(f"Test Ticker: {test_ticker}")

# print the original row from m16_df
print("\n ", m16_df[(m16_df['Date'] == test_date) & (m16_df['Ticker'] == test_ticker)][['Date', 'Ticker', 'Adj Close_x']])

# get the row from m16_df
print(f"\nTest Date after 5 days: {test_date_after_5}")
print(m16_df[(m16_df['Date'] == test_date_after_5) & (m16_df['Ticker'] == test_ticker)][['Date', 'Ticker', 'Adj Close_x']])

Test Date: 2014-09-05 00:00:00
Test Ticker: MSFT

             Date Ticker  Adj Close_x
7181 2014-09-05   MSFT    39.395618

Test Date after 5 days: 2014-09-10 00:00:00
           Date Ticker  Adj Close_x
7184 2014-09-10   MSFT    40.193642


In [ ]:
df_buy_T_copy = df_buy_T.copy()

# Apply the function to each date in df_buy_T_copy to get the next valid trading day
df_buy_T_copy['Date after 5 td'] = df_buy_T_copy['Date'].apply(find_next_trading_day)

# Step 3: Merge df_buy_T_copy with m16_df based on Ticker and the new valid trading dates
df_buy_T_copy = pd.merge(
    df_buy_T_copy,
    m16_df[['Date', 'Ticker', 'Adj Close_x']],
    how='left',
    left_on=['Ticker', 'Date after 5 td'],
    right_on=['Ticker', 'Date']
)

# Rename columns for clarity and drop unnecessary columns
df_buy_T_copy.rename(columns={'Adj Close_x': 'Adj Close_x aft 5 td'}, inplace=True)
df_buy_T_copy.drop(columns=['Weekday', 'buy', 'Date after 5 td'], inplace=True)

df_buy_T_copy

,Date_x,Adj Close_x_x,Ticker,cci,Date_y,Adj Close_x_y
0,2014-05-30,34.912762,MSFT,203.518798,2014-06-04,34.384052
1,2014-09-05,39.395618,MSFT,257.013330,2014-09-10,40.193642
2,2015-04-24,41.630741,MSFT,340.009871,2015-04-29,42.665649
3,2015-10-02,40.151123,MSFT,206.125828,2015-10-07,41.234852
4,2015-10-23,46.583046,MSFT,431.107024,2015-10-28,47.561062
...,...,...,...,...,...,...
455,2022-11-25,2058.108887,LT.NS,210.797547,2022-11-30,2070.181641
456,2023-06-30,2469.979980,LT.NS,219.106421,2023-07-05,2465.240723
457,2023-07-21,2580.430908,LT.NS,359.236523,2023-07-26,2640.295898
458,2023-09-08,2901.600098,LT.NS,303.781710,2023-09-13,2906.750000


In [ ]:
df_buy_T_copy['Profit'] = (df_buy_T_copy['Adj Close_x_y'] - df_buy_T_copy['Adj Close_x_x']) * (1000 / df_buy_T_copy['Adj Close_x_x'])
df_buy_T_copy

,Date_x,Adj Close_x_x,Ticker,cci,Date_y,Adj Close_x_y,Profit
0,2014-05-30,34.912762,MSFT,203.518798,2014-06-04,34.384052,-15.143729
1,2014-09-05,39.395618,MSFT,257.013330,2014-09-10,40.193642,20.256649
2,2015-04-24,41.630741,MSFT,340.009871,2015-04-29,42.665649,24.859233
3,2015-10-02,40.151123,MSFT,206.125828,2015-10-07,41.234852,26.991245
4,2015-10-23,46.583046,MSFT,431.107024,2015-10-28,47.561062,20.995104
...,...,...,...,...,...,...,...
455,2022-11-25,2058.108887,LT.NS,210.797547,2022-11-30,2070.181641,5.865945
456,2023-06-30,2469.979980,LT.NS,219.106421,2023-07-05,2465.240723,-1.918743
457,2023-07-21,2580.430908,LT.NS,359.236523,2023-07-26,2640.295898,23.199610
458,2023-09-08,2901.600098,LT.NS,303.781710,2023-09-13,2906.750000,1.774849


In [ ]:
total_profit = df_buy_T_copy['Profit'].sum()
print(f'Total profit after 5 trading days: ${total_profit:.2f}')

Total profit after 5 trading days: $1041.49


# 5: Finding Customized Strategy for IPOs


Investing inherently involves risk, which essentially means understanding the likelihood of facing a loss. This understanding brings up two critical questions: What is the probability that an investment will reach a loss threshold prompting an exit, and how long will it take to potentially see a return?

These questions become particularly challenging for companies that have just completed an Initial Public Offering (IPO). Due to incomplete disclosures, our analysis must often rely on basic, available financial information. This includes current and projected profits, existing and anticipated debts, and the company’s market share—though the reliability of market share as an indicator can be contentious.

Furthermore, assessing the likelihood of a company avoiding losses involves evaluating traditional valuation metrics. These metrics consider factors like the company's growth potential and whether its structural organization meets the requirements of its development stage. This evaluation helps establish a baseline understanding of the company's financial stability.

The complexity increases when managing stocks post-IPO, as investment strategies must adapt to varying investor expectations and market conditions. Longitudinal data, covering extensive periods, is crucial for this dynamic strategy. Without such data, investors must rely on basic analyses and comparisons with established companies known for their consistent positive returns. This approach provides a framework for relative assessment and informed decision-making in the volatile IPO landscape.

The content below is the details:

## Quantitative Assessments:
- **Financial Health**: Analyzing a company's profitability, projected earnings, existing and potential debts are crucial. Financial statements offer a glimpse into the company’s operational efficiency and financial stability.
- **Market Position**: Understanding the company's market share within its industry provides insights into its competitive edge and market acceptance.

## Qualitative Assessments:
- **Company Structure and Growth Potential**: Evaluating the company’s business model, growth potential, and management team's expertise can indicate its long-term viability.
- **Sector Performance**: Different sectors may show varying results in IPO performance, influenced by market trends and economic conditions.

### Strategic Steps for IPO Investment

#### 1. **Historical Analysis**
Investigate past IPO performances, especially those within the target company’s sector. This analysis should include:
- **Pricing Strategy**: Examining the IPO’s initial price range and subsequent market performance.
- **Post-IPO Performance**: Tracking price fluctuations over different periods post-IPO.

#### 2. **Scoring System**
Create a model to score IPOs based on various factors such as financial stability, growth metrics, market sentiment, and leadership quality. This helps in quantifying potential investments and setting thresholds for investment decisions.

#### 3. **Comparative Market Analysis**
Benchmarking against existing players in the market can provide a relative measure of the IPO’s potential. Comparing metrics like P/E ratios, market caps, and growth rates offers a standard by which to gauge new market entrants.

#### 4. **Strategy**
Adopt a flexible investment approach:
- **Regular Reviews**: Continuously monitor the market and adjust the investment strategy based on performance and market changes.
- **Diversification**: Reduce risks by diversifying investments across various sectors and IPOs.

#### 5. **Public Data**
Leverage public resources such as SEC filings, financial news portals, and stock market databases to gather comprehensive data about upcoming IPOs.